<a href="https://colab.research.google.com/github/Margarita215729/scientific-api/blob/main/MainDataSetsManager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Galaxy Data Aggregation Notebook
_Сбор и объединение реальных данных галактик из обзоров JWST, SDSS DR17, DESI EDR и Euclid Q1 в единый CSV_

**Цель:**  
- Получить максимально полный набор галактик (RA, Dec, z) из реальных наблюдений  
- Привести данные к единому формату и сохранить промежуточные CSV  
- Объединить все данные в один итоговый файл  
- Работать в среде Google Colab с ограничениями ~2.7 GB RAM, ~107 GB диск  
- Данные охватывают z от 0 до ~3.5+  

In [2]:
!pip install numpy pandas astropy

import sys
import time
import logging
import warnings
import csv
import json
import math
import random
from datetime import datetime
import urllib.request
import urllib.error
import urllib.parse
import io
import re
import os
import requests
import numpy as np
import pandas as pd
from astropy.io import fits, ascii
from astropy.table import Table
from astropy import units as u
from astropy.cosmology import Planck18 as cosmo
import sys
import time
import logging
import warnings
import csv
import json
import math
import random
from datetime import datetime
import urllib.request
import urllib.error
import urllib.parse

# Настройка рабочей среды
OUTPUT_DIR = "galaxy_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Настройка логирования
logging.basicConfig(level=logging.INFO,
                   format="%(asctime)s [%(levelname)s] %(message)s",
                   handlers=[
                       logging.StreamHandler(),
                       logging.FileHandler(os.path.join(OUTPUT_DIR, "data_collection.log"))
                   ])
logger = logging.getLogger(__name__)

# Подавление предупреждений
warnings.filterwarnings('ignore')

In [3]:

# Ограничения на количество строк для выборки (чтобы не загружать полностью многогигабайтные файлы)
MAX_ROWS_EUCLID = None    # None = загрузить все
MAX_ROWS_DESI   = None
MAX_ROWS_SDSS   = None
MAX_ROWS_JWST   = None
MAX_ROWS_DES    = 1000000  # по умолчанию ограничим DES Y6 ~1e6 строк (для тестирования и производительности)

CHUNK_SIZE = 1000000  # размер чанка для построчной загрузки больших таблиц (например, DES Y6)
INCLUDE_SIM = False   # включать ли данные симуляции (файл 'galaxy_data/sim.csv'), можно изменить на True при необходимости

# Константы для астрономических вычислений
SPEED_OF_LIGHT = 299792.458  # км/с
H0 = 67.74  # Постоянная Хаббла, км/с/Мпк
OM0 = 0.3089  # Плотность материи

def find_column_name(col_names, options):
    """
    Найти имя колонки в списке col_names, соответствующее (без учета регистра) одному из вариантов в options.
    Возвращает реальное имя колонки из col_names или None, если не найдено.
    """
    col_names_upper = [name.upper() for name in col_names]
    for opt in options:
        opt_up = opt.upper()
        if opt_up in col_names_upper:
            idx = col_names_upper.index(opt_up)
            return col_names[idx]
    return None

# Вспомогательные функции
def download_file(url, filename, chunk_size=8192, headers=None):
    """
    Скачивает файл с отображением прогресса

    Parameters:
    url - URL для скачивания
    filename - имя файла для сохранения
    chunk_size - размер блока данных
    headers - дополнительные заголовки

    Returns:
    True, если скачивание успешно, False в противном случае
    """
    if not headers:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
            'Cache-Control': 'max-age=0'
        }

    try:
        logger.info(f"Скачивание файла из {url}")

        # Создаем запрос с заголовками
        req = urllib.request.Request(url, headers=headers)

        # Открываем соединение
        with urllib.request.urlopen(req) as response, open(filename, 'wb') as out_file:
            # Получаем общий размер, если доступен
            content_length = response.getheader('Content-Length')
            total_size = int(content_length) if content_length else None

            downloaded = 0
            start_time = time.time()
            last_update = start_time

            while True:
                chunk = response.read(chunk_size)
                if not chunk:
                    break

                out_file.write(chunk)
                downloaded += len(chunk)

                # Обновляем индикатор прогресса
                current_time = time.time()
                if current_time - last_update > 2 and total_size:
                    percent = downloaded * 100 / total_size
                    elapsed = current_time - start_time
                    speed = downloaded / elapsed / 1024 if elapsed > 0 else 0
                    logger.info(f"Загружено {downloaded/(1024*1024):.1f} MB / {total_size/(1024*1024):.1f} MB ({percent:.1f}%) скорость: {speed:.1f} KB/s")
                    last_update = current_time

        logger.info(f"Файл успешно загружен и сохранен как {filename}")
        return True

    except urllib.error.HTTPError as e:
        logger.error(f"HTTP ошибка при скачивании {url}: {e.code} {e.reason}")
        return False
    except urllib.error.URLError as e:
        logger.error(f"URL ошибка при скачивании {url}: {e.reason}")
        return False
    except Exception as e:
        logger.error(f"Ошибка при скачивании {url}: {str(e)}")
        return False

def rad_to_deg(rad):
    """Перевод радиан в градусы"""
    return rad * 180.0 / math.pi

def deg_to_rad(deg):
    """Перевод градусов в радианы"""
    return deg * math.pi / 180.0

def e_z(z, Om0):
    """Функция E(z) для плоской ΛCDM модели"""
    return math.sqrt(Om0 * (1 + z)**3 + (1 - Om0))

def comoving_distance(z, H0=H0, Om0=OM0):
    """
    Вычисляет комовское расстояние для заданного красного смещения

    Parameters:
    z - красное смещение
    H0 - постоянная Хаббла, км/с/Мпк
    Om0 - плотность материи

    Returns:
    Комовское расстояние в Мпк
    """
    # Проверка входных данных
    try:
        z = float(z)
        if z < 0:
            logger.warning(f"Отрицательное красное смещение: {z}, используем |z|")
            z = abs(z)
        elif z > 20:
            logger.warning(f"Слишком большое красное смещение: {z}, ограничиваем до 20")
            z = 20.0
    except (ValueError, TypeError):
        logger.warning(f"Некорректное красное смещение: {z}, используем z=1.0")
        z = 1.0

    # Для малых z используем приближение
    if z < 0.01:
        return SPEED_OF_LIGHT * z / H0

    # Для больших z используем численное интегрирование
    n_steps = 100
    dz = z / n_steps
    integral = 0

    for i in range(n_steps):
        z_i = i * dz
        integral += 1 / e_z(z_i, Om0) * dz

    # Умножаем на c/H0
    return SPEED_OF_LIGHT / H0 * integral

def sky_to_cartesian(ra, dec, z):
    """
    Преобразует астрономические координаты в декартовы

    Parameters:
    ra, dec - прямое восхождение и склонение в градусах
    z - красное смещение

    Returns:
    x, y, z - декартовы координаты в Мпк или None в случае ошибки
    """
    try:
        # Проверка входных данных
        ra = float(ra)
        dec = float(dec)
        z = float(z)

        # Нормализация значений
        if ra < 0 or ra > 360:
            ra = ra % 360
        if dec < -90 or dec > 90:
            dec = max(-90, min(90, dec))
        if z <= 0:
            logger.warning(f"Некорректное красное смещение: {z}, используем z=0.1")
            z = 0.1

        # Перевод в радианы
        ra_rad = deg_to_rad(ra)
        dec_rad = deg_to_rad(dec)

        # Комовское расстояние
        dist = comoving_distance(z)

        # Перевод в декартовы координаты
        x = dist * math.cos(dec_rad) * math.cos(ra_rad)
        y = dist * math.cos(dec_rad) * math.sin(ra_rad)
        z_cart = dist * math.sin(dec_rad)

        return x, y, z_cart

    except (ValueError, TypeError) as e:
        logger.warning(f"Ошибка при преобразовании координат: {str(e)} (ra={ra}, dec={dec}, z={z})")
        return None

def parse_csv_data(csv_content, delimiter=',', skip_lines=0):
    """
    Парсит CSV данные и возвращает список словарей

    Parameters:
    csv_content - содержимое CSV файла (строка)
    delimiter - разделитель полей
    skip_lines - количество строк для пропуска в начале

    Returns:
    Список словарей с данными
    """
    try:
        # Пропускаем начальные строки, если нужно
        lines = csv_content.splitlines()
        if skip_lines > 0:
            if len(lines) <= skip_lines:
                logger.warning(f"CSV содержит только {len(lines)} строк, но нужно пропустить {skip_lines}")
                return []
            lines = lines[skip_lines:]

        # Удаляем пустые строки
        lines = [line.strip() for line in lines if line.strip()]

        if not lines:
            logger.warning("CSV не содержит данных")
            return []

        # Читаем CSV
        csv_reader = csv.DictReader(io.StringIO('\n'.join(lines)), delimiter=delimiter)
        data = [row for row in csv_reader]

        logger.info(f"Прочитано {len(data)} строк из CSV")
        return data

    except Exception as e:
        logger.error(f"Ошибка при парсинге CSV: {str(e)}")
        return []

def save_data_to_csv(data, filename, fieldnames=None):
    """
    Сохраняет список словарей в CSV файл

    Parameters:
    data - список словарей
    filename - путь к файлу
    fieldnames - список имен полей (столбцов)

    Returns:
    True если успешно, False в случае ошибки
    """
    try:
        if not data:
            logger.warning("Нет данных для сохранения")
            return False

        # Определяем имена полей
        if not fieldnames:
            # Находим все возможные ключи в данных
            all_keys = set()
            for row in data:
                all_keys.update(row.keys())

            # Исключаем пустые ключи
            all_keys = [key for key in all_keys if key]

            # Приоритетные поля в начале списка
            priority_fields = ['ra', 'dec', 'z', 'source', 'x', 'y', 'z_cart']
            fieldnames = [field for field in priority_fields if field in all_keys]
            fieldnames.extend([field for field in all_keys if field not in priority_fields])

        # Проверяем данные перед сохранением
        valid_data = []
        for row in data:
            # Создаем новую запись только с полями из fieldnames
            valid_row = {}
            for field in fieldnames:
                if field in row:
                    valid_row[field] = row[field]

            valid_data.append(valid_row)

        # Сохраняем данные
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(valid_data)

        logger.info(f"Сохранено {len(valid_data)} строк в {filename}")
        return True

    except Exception as e:
        logger.error(f"Ошибка при сохранении CSV: {str(e)}")
        return False

def load_data_from_csv(filename):
    """
    Загружает данные из CSV файла в список словарей

    Parameters:
    filename - путь к файлу

    Returns:
    Список словарей с данными
    """
    try:
        data = []
        with open(filename, 'r', newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                # Преобразуем строковые значения в числа, где нужно
                numeric_fields = ['ra', 'dec', 'z', 'x', 'y', 'z_cart']
                for field in numeric_fields:
                    if field in row and row[field]:
                        try:
                            row[field] = float(row[field])
                        except ValueError:
                            pass
                data.append(row)

        logger.info(f"Загружено {len(data)} строк из {filename}")
        return data

    except FileNotFoundError:
        logger.error(f"Файл не найден: {filename}")
        return []
    except Exception as e:
        logger.error(f"Ошибка при загрузке CSV: {str(e)}")
        return []


def get_euclid_data():
    EUCLID_URL = "https://irsa.ipac.caltech.edu/ibe/data/euclid/q1/catalogs/MER_FINAL_CATALOG/102018211/EUC_MER_FINAL-CAT_TILE102018211-CC66F6_20241018T214045.289017Z_00.00.fits"
    """Загрузить Euclid Q1 MER Final (FITS) и сконвертировать в CSV."""
    output_path = os.path.join(OUTPUT_DIR, "euclid.csv")
    if os.path.exists(output_path):
        print("Euclid: CSV уже есть — пропускаем.")

    print("Euclid: поиск HDU и чтение таблицы...")
    # пытаемся найти в FITS бинарную таблицу с данными
    tbl = None
    for ext in [1, 2, 3]:
        try:
            candidate = Table.read(EUCLID_URL, hdu=ext)
            if len(candidate) > 10:
                tbl = candidate
                print(f"  → HDU={ext}, строк={len(tbl)}")
                break
        except Exception:
            continue
    if tbl is None:
        raise RuntimeError("Euclid: не удалось найти HDU с данными")


    # Определяем колонки RA, DEC, REDSHIFT
    cols = tbl.colnames
    ra_col  = find_column_name(cols, ["RA","RAJ2000","ALPHA_J2000","ra_deg","ra","Ra","ra(deg)"])
    dec_col = find_column_name(cols, ["DEC","DECJ2000","DEJ2000","delta_j2000","dec_deg","dec","Dec","dec(deg)"])
    z_col   = find_column_name(cols, ["Z","REDSHIFT","Z_SPEC","z_spec","PHOTOZ","redshift","z","z_phot","Redshift"])
    if not ra_col or not dec_col:
        raise RuntimeError(f"Euclid: не найдены RA/DEC в {cols}")

    # Красное смещение в MER Final обычно отсутствует — заполняем NaN
    z_col = find_column_name(cols, ["REDSHIFT", "Z", "Z_SPEC", "Z_SPEC_PHOT", "PHOTOZ", "z_phot"])
    ra_data  = np.array(tbl[ra_col])
    dec_data = np.array(tbl[dec_col])
    z_data   = np.array(tbl[z_col]) if z_col else np.full(len(tbl), np.nan)

    # Собираем DataFrame и обрезаем, если надо
    df = pd.DataFrame({"RA": ra_data, "DEC": dec_data, "redshift": z_data})
    if MAX_ROWS_EUCLID is not None and len(df) > MAX_ROWS_EUCLID:
        df = df.iloc[:MAX_ROWS_EUCLID]

    # Сохраняем
    df.to_csv(output_path, index=False)
    print(f"Euclid: сохранено {len(df)} объектов → {output_path}")
    return True


def get_sdss_data():
    """Загрузить SDSS DR17 spectro (FITS) и сконвертировать в CSV."""
    SDSS_URL = "https://data.sdss.org/sas/dr17/sdss/spectro/redux/specObj-dr17.fits"
    output_path = os.path.join(OUTPUT_DIR, "sdss.csv")
    if os.path.exists(output_path):
        print("SDSS: CSV уже есть — пропускаем.")
        return output_path

    print("SDSS: поиск HDU и чтение таблицы...")
    tbl = None
    for ext in [1,2,3]:
        try:
            candidate = Table.read(SDSS_URL, hdu=ext)
            if len(candidate) > 10:
                tbl = candidate
                print(f"SDSS: выбрана HDU={ext}, строк={len(tbl)}")
                break
        except Exception:
            continue
    if tbl is None:
        raise RuntimeError("SDSS: не удалось найти подходящий HDU с данными")

    cols = tbl.colnames
    ra_col  = find_column_name(cols, ["RA","RAJ2000","ALPHA_J2000","ra_deg"])
    dec_col = find_column_name(cols, ["DEC","DECJ2000","DEJ2000","delta_j2000","dec_deg"])
    z_col   = find_column_name(cols, ["Z","REDSHIFT","Z_NOQSO","Z_SPEC","PHOTOZ","redshift"])

    data = {
        "RA": np.array(tbl[ra_col]),
        "DEC": np.array(tbl[dec_col]),
        "redshift": np.array(tbl[z_col]) if z_col else np.full(len(tbl), np.nan)
    }
    df = pd.DataFrame(data)
    if MAX_ROWS_SDSS and len(df) > MAX_ROWS_SDSS:
        df = df.iloc[:MAX_ROWS_SDSS]

    df.to_csv(output_path, index=False)
    print(f"SDSS: сохранено {len(df)} объектов → {output_path}")
    return True



def get_desi_data():
  DESI_URL = "https://data.desi.lbl.gov/public/dr1/survey/catalogs/dr1/LSS/iron/LSScats/v1.2/ELG_LOPnotqso_NGC_clustering.dat.fits"
  output_path = os.path.join(OUTPUT_DIR, "desi.csv")
  if os.path.exists(output_path):
      print("Файл DESI (desi.csv) уже существует, пропуск загрузки.")
      return output_path
  print("Скачивание данных DESI DR1 ELG clustering...")
  hdul = fits.open(DESI_URL, memmap=True)
  data = hdul[1].data
  # Имена колонок
  ra_col  = find_column_name(hdul[1].columns.names, ["RA", "RAJ2000", "ALPHA_J2000"])
  dec_col = find_column_name(hdul[1].columns.names, ["DEC", "DECJ2000", "DEJ2000", "DELTA_J2000"])
  z_col   = find_column_name(hdul[1].columns.names, ["Z", "REDSHIFT", "Z_SPEC", "ZSPEC", "Z_PHOT", "ZPHOT", "PHOTOZ", "Z_MEAN"])
  ra_data = data[ra_col]
  dec_data = data[dec_col]
  if z_col:
      z_data = data[z_col]
  else:
      z_data = np.full(len(ra_data), np.nan)
    # Обработка масок
  if hasattr(ra_data, 'mask'):
      ra_data = ra_data.filled(np.nan)
  if hasattr(dec_data, 'mask'):
      dec_data = dec_data.filled(np.nan)
  if hasattr(z_data, 'mask'):
      z_data = z_data.filled(np.nan)
  df = pd.DataFrame({"RA": ra_data, "DEC": dec_data, "redshift": z_data})
  if MAX_ROWS_DESI is not None and len(df) > MAX_ROWS_DESI:
      df = df.iloc[:MAX_ROWS_DESI]

  df.to_csv(output_path, index=False)
  hdul.close()
  print(f"DESI: сохранено объектов: {len(df)}")
  return True

def get_jwst_data():
  ceers_url = "https://web.corral.tacc.utexas.edu/ceersdata/DR06/MIRI/miri_catalog.dat"
  """Загрузить каталог JWST CEERS PR1 (MIRI) и сохранить в CSV."""
  output_path = os.path.join(OUTPUT_DIR, "jwst.csv")
  if os.path.exists(output_path):
      print("Файл JWST (jwst.csv) уже существует, пропуск загрузки.")
      return output_path
  print("Скачивание данных JWST CEERS PR1 MIRI...")
  # Загружаем текстовый .dat файл
  response = requests.get(JWST_URL)
  response.raise_for_status()
  content = response.text
  # Читаем содержимое как таблицу ASCII с помощью astropy
  table = ascii.read(content)
  # Имена колонок (может быть 'RA' и 'Dec' или подобные)
  col_names = table.colnames
  ra_col  = find_column_name(col_names, ["RA", "RAJ2000", "ALPHA_J2000"])
  dec_col = find_column_name(col_names, ["Dec", "DEC", "DEJ2000", "Delta_J2000"])
  z_col   = find_column_name(col_names, ["Z", "REDSHIFT", "Z_SPEC", "Z_PHOT", "PHOTOZ"])
  ra_data = table[ra_col]
  dec_data = table[dec_col]
  if z_col:
      z_data = table[z_col]
  else:
      z_data = np.full(len(ra_data), np.nan)
  # Конвертируем в DataFrame
  df = pd.DataFrame({ "RA": np.array(ra_data), "DEC": np.array(dec_data), "redshift": np.array(z_data) })
  if MAX_ROWS_JWST is not None and len(df) > MAX_ROWS_JWST:
      df = df.iloc[:MAX_ROWS_JWST]
  df.to_csv(output_path, index=False)
  print(f"JWST: сохранено объектов: {len(df)}")
  return True


def get_des_data():
    DES_URL    = "http://desdr-server.ncsa.illinois.edu/despublic/Y6_GOLD_v2.0.fits"
    """Загрузить каталог DES Y6 GOLD и сохранить в CSV (построчная обработка из-за большого объема)."""
    output_path = os.path.join(OUTPUT_DIR, "des.csv")
    if os.path.exists(output_path):
        print("Файл DES (des.csv) уже существует, пропуск загрузки.")
        return output_path
    print("Скачивание данных DES Y6 GOLD (может занять время)...")
    hdul = fits.open(DES_URL, memmap=True)
    data_hdu = hdul[1]
    # Определяем имена колонок
    ra_col  = find_column_name(data_hdu.columns.names, ["RA", "RAJ2000", "ALPHA_J2000"])
    dec_col = find_column_name(data_hdu.columns.names, ["DEC", "DECJ2000", "DEJ2000", "DELTA_J2000"])
    z_col   = find_column_name(data_hdu.columns.names, ["Z", "REDSHIFT", "PHOTOZ", "Z_MEAN", "Z_SPEC", "ZPHOT"])
    # Получаем общее число строк и применяем ограничение MAX_ROWS_DES
    total_rows = data_hdu.header.get('NAXIS2', None)
    if total_rows is None:
        total_rows = len(data_hdu.data)  # на случай, если NAXIS2 недоступен
    if MAX_ROWS_DES is not None and total_rows > MAX_ROWS_DES:
        target_rows = MAX_ROWS_DES
    else:
        target_rows = total_rows
    # Открываем выходной файл и пишем заголовок
    with open(output_path, 'w', encoding='utf-8') as f_out:
        f_out.write("RA,DEC,redshift\n")
    # Читаем и сохраняем данные чанками, чтобы не загружать все в память сразу
    rows_processed = 0
    for start in range(0, target_rows, CHUNK_SIZE):
        stop = min(target_rows, start + CHUNK_SIZE)
        data_chunk = data_hdu.data[start:stop]  # извлекаем срез данных
        ra_data = data_chunk[ra_col]
        dec_data = data_chunk[dec_col]
        if z_col:
            z_data = data_chunk[z_col]
        else:
            z_data = np.full(len(ra_data), np.nan)
        # Обрабатываем маски при наличии
        if hasattr(ra_data, 'mask'):
            ra_data = ra_data.filled(np.nan)
        if hasattr(dec_data, 'mask'):
            dec_data = dec_data.filled(np.nan)
        if hasattr(z_data, 'mask'):
            z_data = z_data.filled(np.nan)
        # Формируем DataFrame для чанка и дописываем в CSV
        chunk_df = pd.DataFrame({"RA": ra_data, "DEC": dec_data, "redshift": z_data})
        # Режим 'a' добавляет строки, header=False чтобы не дублировать заголовок
        chunk_df.to_csv(output_path, mode='a', header=False, index=False)
        rows_processed += len(chunk_df)
        print(f"DES: обработано {rows_processed} из ~{target_rows} объектов...", end="\r")
    hdul.close()
    print(f"\nDES: сохранено объектов: {rows_processed}")
    return True

def merge_all_data():
    """
    Объединить все каталоги в единый набор и преобразовать сферические координаты (RA, DEC, redshift) в декартовые (X, Y, Z).
    """
    # Пути ко всем подготовленным CSV-файлам
    data_files = [
        (os.path.join(OUTPUT_DIR, "euclid.csv"), "Euclid"),
        (os.path.join(OUTPUT_DIR, "desi.csv"),   "DESI"),
        (os.path.join(OUTPUT_DIR, "sdss.csv"),   "SDSS"),
        (os.path.join(OUTPUT_DIR, "jwst.csv"),   "JWST"),
        (os.path.join(OUTPUT_DIR, "des.csv"),    "DES")
    ]
    # Открываем выходной файл для объединенных данных
    merged_path = os.path.join(OUTPUT_DIR, "all_data.csv")
    with open(merged_path, 'w', encoding='utf-8') as f_out:
        f_out.write("survey,RA,DEC,redshift,X,Y,Z\n")
    total_count = 0
    # Обрабатываем каждый каталог по очереди, чтобы не хранить все данные одновременно в памяти
    for file_path, survey_name in data_files:
        if not os.path.exists(file_path):
            print(f"Предупреждение: файл {file_path} не найден, пропуск.")
            continue
        print(f"Объединение данных: {survey_name}")
        # Читаем входной CSV чанками, чтобы обработка была поэтапной
        for chunk in pd.read_csv(file_path, chunksize=CHUNK_SIZE):
            # Добавляем колонку с названием обзора
            chunk.insert(0, 'survey', survey_name)
            # Переводим координаты в радианы
            ra_rad = np.deg2rad(chunk["RA"].values)
            dec_rad = np.deg2rad(chunk["DEC"].values)
            z_vals = chunk["redshift"].values
            # Вычисляем комовскую дистанцию (в Mpc) для каждого redshift (для NaN останется NaN)
            distances = np.full(z_vals.shape, np.nan)
            mask = np.isfinite(z_vals)  # маска для имеющих определённый z
            if mask.any():
                distances[mask] = cosmo.comoving_distance(z_vals[mask]).to(u.Mpc).value
            # Расчет декартовых координат
            x = distances * np.cos(dec_rad) * np.cos(ra_rad)
            y = distances * np.cos(dec_rad) * np.sin(ra_rad)
            z_cart = distances * np.sin(dec_rad)
            # Добавляем колонки X, Y, Z в chunk
            chunk["X"] = x
            chunk["Y"] = y
            chunk["Z"] = z_cart
            # Дописываем chunk в объединенный CSV
            chunk.to_csv(merged_path, mode='a', header=False, index=False)
            total_count += len(chunk)
    print(f"Объединение завершено. Общее число объектов: {total_count}")


def main():
    """
    Главная функция скрипта: последовательно получает и объединяет все данные
    """
    logger.info("=== Начало сбора данных галактик ===")

    # Получение данных из всех источников
    success = True
    success = get_euclid_data()
    success = get_sdss_data()
    success = get_desi_data()
    success = get_jwst_data()

    if not success:
        logger.warning("Некоторые источники данных не были получены.")

    # Объединение и преобразование
    if merge_all_data():
        logger.info("=== Завершено успешно: все данные объединены и обработаны ===")
    else:
        logger.error("Произошла ошибка при объединении данных.")

if __name__ == "__main__":
    main()


Euclid: поиск HDU и чтение таблицы...
  → HDU=1, строк=465


RuntimeError: Euclid: не найдены RA/DEC в ['OBJECT_ID', 'RIGHT_ASCENSION', 'DECLINATION', 'RIGHT_ASCENSION_PSF_FITTING', 'DECLINATION_PSF_FITTING', 'SEGMENTATION_MAP_ID', 'VIS_DET', 'FLUX_VIS_1FWHM_APER', 'FLUX_VIS_2FWHM_APER', 'FLUX_VIS_3FWHM_APER', 'FLUX_VIS_4FWHM_APER', 'FLUX_Y_1FWHM_APER', 'FLUX_Y_2FWHM_APER', 'FLUX_Y_3FWHM_APER', 'FLUX_Y_4FWHM_APER', 'FLUX_J_1FWHM_APER', 'FLUX_J_2FWHM_APER', 'FLUX_J_3FWHM_APER', 'FLUX_J_4FWHM_APER', 'FLUX_H_1FWHM_APER', 'FLUX_H_2FWHM_APER', 'FLUX_H_3FWHM_APER', 'FLUX_H_4FWHM_APER', 'FLUX_NIR_STACK_1FWHM_APER', 'FLUX_NIR_STACK_2FWHM_APER', 'FLUX_NIR_STACK_3FWHM_APER', 'FLUX_NIR_STACK_4FWHM_APER', 'FLUX_U_EXT_DECAM_1FWHM_APER', 'FLUX_U_EXT_DECAM_2FWHM_APER', 'FLUX_U_EXT_DECAM_3FWHM_APER', 'FLUX_U_EXT_DECAM_4FWHM_APER', 'FLUX_G_EXT_DECAM_1FWHM_APER', 'FLUX_G_EXT_DECAM_2FWHM_APER', 'FLUX_G_EXT_DECAM_3FWHM_APER', 'FLUX_G_EXT_DECAM_4FWHM_APER', 'FLUX_R_EXT_DECAM_1FWHM_APER', 'FLUX_R_EXT_DECAM_2FWHM_APER', 'FLUX_R_EXT_DECAM_3FWHM_APER', 'FLUX_R_EXT_DECAM

### 1. Установка зависимостей и настройка окружения

In [ ]:
# 1. Установка зависимостей в Google Colab
!pip install --quiet astroquery sdss-access pyvo astropy requests pyvo.dal astroquery.mast desilike

#!pip install --quiet git+https://github.com/cosmodesi/desilike.git
!pip install --upgrade --force-reinstall numpy==1.26.0 pandas==2.2.2 #right ver
# 2. Импорт библиотек и настройка окружения
import os
import logging

import numpy as np
import pandas as pd
import requests

from astropy import units as u
from astropy.cosmology import Planck15
from astropy.table import Table

# SDSS через sdss-access
from sdss_access import Access

# DESI TAP-запросы
from pyvo.dal import TAPService

# JWST (если понадобится)
from astroquery.mast import Observations

# Euclid через IRSA
from astroquery.ipac.irsa import Irsa

# Настройка логирования
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)
logger.info("Окружение готово: зависимости установлены и библиотеки импортированы.")


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### 2. Загрузка и фильтрация данных SDSS DR17

**Источник:** SDSS DR17 specObj CSV (~2 GB .bz2)  
**Задача:** оставить только внегалактические объекты (class≠STAR, zwarning=0) и сохранить столбцы ra, dec, z

In [ ]:
# 2.1. Скачиваем сжатый CSV
sdss_url = "https://dr17.sdss.org/sas/dr17/casload/spCSV/plates/sqlSpecObj.csv.bz2"
sdss_bz2 = "sqlSpecObj_dr17.csv.bz2"
if not os.path.exists(sdss_bz2):
    logger.info("Скачивание SDSS DR17...")
    r = requests.get(sdss_url, stream=True)
    r.raise_for_status()
    with open(sdss_bz2, "wb") as f:
        for chunk in r.iter_content(8192):
            f.write(chunk)
    logger.info("Загрузка завершена.")

import pandas as pd

# Просмотрим имена колонок в начале файла
header = pd.read_csv("sqlSpecObj_dr17.csv.bz2", compression="bz2", nrows=0)
print("Columns in SDSS CSV:", header.columns.tolist())

# 2.2. Фильтруем по чанкам
sdss_out = "sdss.csv"
with open(sdss_out, "w") as f:
    f.write("ra,dec,z\n")

cols = ["ra","dec","z","class","zWarning"]
chunksize = 100_000
total = 0
for chunk in pd.read_csv(sdss_bz2, compression="bz2", usecols=cols, chunksize=chunksize):
    mask = (chunk["class"].str.upper() != "STAR") & (chunk["zWarning"] == 0)
    df = chunk.loc[mask, ["ra","dec","z"]]
    df.to_csv(sdss_out, mode="a", header=False, index=False)
    total += len(df)
    logger.info(f"SDSS: добавлено {len(df)} записей (итого {total})")

logger.info(f"SDSS-фильтрация завершена. Всего объектов: {total}")


[ERROR]: Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-6750489cf23f>", line 9, in <cell line: 0>
    for chunk in r.iter_content(8192):
  File "/usr/local/lib/python3.11/dist-packages/requests/models.py", line 820, in generate
    yield from self.raw.stream(chunk_size, decode_content=True)
  File "/usr/local/lib/python3.11/dist-packages/urllib3/response.py", line 1066, in stream
    data = self.read(amt=amt, decode_content=decode_content)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/response.py", line 955, in read
    data = self._raw_read(amt)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/response.py", line 879, in _raw_read
    data = self._fp_read(amt, read1=read1) if not fp_closed else b""
          

### 3. Загрузка и фильтрация данных DESI EDR

**Источник:** DESI EDR через TAP (NOIRLab Data Lab)  
**Таблицы:** desi_edr.zpix (z, zwarn, spectype), desi_edr.photometry (ra, dec)  
**Фильтр:** zwarn=0, spectype≠'STAR'

In [ ]:
from pyvo.dal import TAPService

tap = TAPService("https://datalab.noirlab.edu/tap")
query = """
SELECT p.ra, p.dec, z.z
FROM desi_edr.zpix AS z
JOIN desi_edr.photometry AS p ON z.targetid = p.targetid
WHERE z.zwarn=0 AND z.spectype<>'STAR'
"""

logger.info("Отправка TAP-запроса к DESI EDR...")
job = tap.submit_job(query)
job.run().wait()
table = job.fetch_result()
df_desi = table.to_table().to_pandas()
desi_out = "desi.csv"
df_desi.to_csv(desi_out, index=False)
logger.info(f"DESI EDR: получено {len(df_desi)} объектов и сохранено в {desi_out}")


ModuleNotFoundError: No module named 'pyvo'

### 4. Загрузка и фильтрация данных Euclid Q1


**Источник:** Euclid Q1 через IRSA (astroquery.ipac.Irsa)  
**Каталог:** euclid_q1_mer_catalog (содержит photoz и/или specz)  
**Поиск:** конус вокруг центрального поля (для примера), затем можно масштабировать


In [ ]:
from astroquery.irsa import Irsa

# Получим список всех доступных каталогов с «euclid» в имени
catalogs = Irsa.list_catalogs(filter="euclid")
print(catalogs)


In [ ]:
# 4. Загрузка и фильтрация данных Euclid Q1 через TAP-запрос к IRSA

from astroquery.ipac.irsa import Irsa
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table

# Список полей: (название, RA, Dec, радиус)
regions = [
    ("EDFS", 34.5, -4.5, 0.5),
    ("Euclid Deep Field North", 34.0, 0.0, 0.5),
    ("CEERS", 150.0, 2.2, 0.5),
    ("Fornax", 83.8, -5.4, 0.5)
]

# Инициализируем общий результирующий каталог
combined_table = Table(names=('ra','dec','z'), dtype=('f8','f8','f8'))

total_count = 0
for name, ra, dec, radius in regions:
    # Создаем объект SkyCoord для центра области
    coord = SkyCoord(ra, dec, unit='deg')
    try:
        # Выполняем конусный запрос к каталогу euclid_q1_mer_catalogue
        table = Irsa.query_region(coord, catalog="euclid_q1_mer_catalogue",
                                   spatial="Cone", radius=radius * u.deg)
    except Exception as e:
        print(f"Ошибка запроса для области {name}: {e}")
        continue  # пропускаем эту область в случае ошибки

    if table is None or len(table) == 0:
        print(f"{name}: Нет данных в указанной области (0 результатов).")
        continue
    # Ищем колонку с фотометрическим z (photo-z) без учета регистра
    z_col = None
    for col in table.colnames:
        if col.lower().startswith("photoz") or col.lower() == "z":
            z_col = col
            break

    if z_col is None:
        print(f"{name}: Колонка с photometric z не найдена в результатах.")
        continue

    # Фильтруем строки с непустым значением фотометрического z
    col_data = table[z_col]
    if hasattr(col_data, 'mask'):  # если колонка маскирована (MaskedColumn)
        valid_rows = ~col_data.mask  # True для тех, где не маскировано
    else:
        # для обычной колонки проверяем на NaN (для числовых типов)
        valid_rows = np.isfinite(col_data)

    filtered_table = table[valid_rows]

    # Добавляем только столбцы ra, dec и z (photo-z) в объединенную таблицу
    # Переименовываем колонку фотометрического z в 'z' для вывода
    subset = filtered_table[['ra', 'dec', z_col]].copy()
    subset.rename_column(z_col, 'z')
    combined_table = Table.vstack([combined_table, subset])  # накапливаем результаты

    # Логирование количества объектов
    count = len(subset)
    total_count += count
    print(f"{name}: найдено {count} объектов с photo-z (накопительно: {total_count}).")

# Сохраняем объединенные результаты в CSV-файл
combined_table.write("euclid.csv", format="ascii.csv", overwrite=True)
print(f"Итого сохранено объектов: {total_count}")


### 5. Объединение промежуточных CSV и вычисление Cartesian coords

Склеиваем sdss.csv, desi.csv, euclid.csv в один all_galaxies.csv,
затем добавляем столбцы x,y,z_cart (комовские координаты в Mpc).

In [ ]:
# 5.1. Чтение и объединение
files = ["sdss.csv", "desi.csv", "euclid.csv"]
dfs = []
for fn in files:
    logger.info(f"Чтение {fn}...")
    df = pd.read_csv(fn)
    dfs.append(df)
df_all = pd.concat(dfs, ignore_index=True)
logger.info(f"Объединено всего {len(df_all)} записей.")

# 5.2. Вычисление комовского расстояния и координат
logger.info("Вычисление комовских координат...")
df_all["dist_mpc"] = Planck15.comoving_distance(df_all["z"]).to_value()
ra_rad  = np.deg2rad(df_all["ra"].values)
dec_rad = np.deg2rad(df_all["dec"].values)
r       = df_all["dist_mpc"].values
df_all["x"] = r * np.cos(dec_rad) * np.cos(ra_rad)
df_all["y"] = r * np.cos(dec_rad) * np.sin(ra_rad)
df_all["z_cart"] = r * np.sin(dec_rad)

# 5.3. Сохранение итогового файла
final_out = "all_galaxies.csv"
df_all.to_csv(final_out, index=False)
logger.info(f"Итоговый файл сохранен: {final_out} (строк: {len(df_all)})")

### 6. Итоговая проверка ресурсов и пример

In [ ]:
# Выведем несколько строк и информацию о памяти
logger.info("Пример первых 5 строк итогового набора:")
print(df_all.head())

# Проверим размер файла на диске
size_gb = os.path.getsize(final_out) / 1e9
logger.info(f"Размер all_galaxies.csv: {size_gb:.2f} GB")


NameError: name 'df_all' is not defined